In [1]:
import bs4
import os
import pandas as pd

import confidential

from datetime import datetime, timedelta

def is_stream_span(el):
    target_class_list = 'TableCell__TableCellElement-sc-1nn7cfv-0 kJgiFu styled__RightTableCell-sc-135veyd-4 kGfYTK'.split(' ')
    if not el.has_attr('class'):
        return False
    return set(el['class']) == set(target_class_list) 

source_dir = "/home/aldakata/Projects/Tuebingen/Data Literacy/cultural-distance-from-spotify-data/spotify_data/"
source_file = "global_2023-01-05.html"

def extract_data(source_dir, source_file):
    """Function to extract data from the html files"""
    with open(f"{source_dir}/{source_file}", "r") as input_file:
        data = input_file.read()
    # Turn the content into soup
    soup = bs4.BeautifulSoup(data, features="html.parser")

    # Find all the rows in the html
    rows = soup.findAll("tr")
    df_rows = []

    for i, row in enumerate(rows[1:]):
        row_url = row.find('a', class_='Button-me270r-0', href=True)['href']
        row_data = {"spotify_url": row_url.replace('https://open.spotify.com/track/', '')}
        week_start = source_file.split(".")[0].split("_")[-1]
        row_data["week_start"] = week_start
        week_start.split("-")
        row_data["streams"] = int(row.find_all(is_stream_span)[0].get_text().replace(',', ''))
        row_data['country'] = source_file.split(".")[0].split("_")[0]
        row_data['ranking'] = i + 1
        df_rows.append(row_data)

    return pd.DataFrame(df_rows)

In [143]:
df = extract_data(source_dir, source_file)
df

,spotify_url,week_start,streams,country,ranking
0,https://open.spotify.com/track/1Qrg8KqiBpW07V7...,2023-01-05,41474366,global,1
1,https://open.spotify.com/track/3nqQXoyQOWXiESF...,2023-01-05,33132175,global,2
2,https://open.spotify.com/track/4Dvkj6JhhA12EX0...,2023-01-05,30998084,global,3
3,https://open.spotify.com/track/4uUG5RXrOk84mYE...,2023-01-05,30704256,global,4
4,https://open.spotify.com/track/2dHHgzDwk4BJdRw...,2023-01-05,28092232,global,5
...,...,...,...,...,...
195,https://open.spotify.com/track/4tYFy8ALRjIZvnv...,2023-01-05,6782777,global,196
196,https://open.spotify.com/track/37F7E7BKEw2E4O2...,2023-01-05,6775767,global,197
197,https://open.spotify.com/track/5g7sDjBhZ4I3gcF...,2023-01-05,6758678,global,198
198,https://open.spotify.com/track/3rb0tMq42Wfgguc...,2023-01-05,6748442,global,199


In [ ]:
# list_df = []
htmls_to_parse = os.listdir(source_dir)
for i, file_path in enumerate(htmls_to_parse):
    if i>1236 and '.html' in file_path:
        print(f'[{i}/{len(htmls_to_parse)}] Parsing {file_path}')
        list_df.append(extract_data(source_dir, file_path))

In [8]:
target_pickle = 'big_boy_df'
df = pd.concat(list_df)
df.to_pickle(f"{source_dir}/{target_pickle}.pkl")  

In [2]:
source_pickle = 'big_boy_df'
df = pd.read_pickle(f'{source_dir}{source_pickle}.pkl')
df = df.reset_index()

In [12]:
source_pickle = 'api_df'
df_api = pd.read_pickle(f'{source_dir}{source_pickle}.pkl')

In [25]:
tracks = []

In [33]:
import numpy as np

total_songs = np.array(list(set(df.spotify_url)))
actual_songs = np.array(list(set(df_api.spotify_url)))


target_urls = []
for s in total_songs:
    if s not in actual_songs:
        target_urls.append(s)


total_to_scrape = len(target_urls)
total_to_scrape

1

In [28]:
pip install progress

Note: you may need to restart the kernel to use updated packages.


In [30]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from time import sleep

cid = confidential.CID
secret = confidential.SECRET

client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret
)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_features(id):
    metadata = sp.track(id)
    features = sp.audio_features(id)

    # metadata
    name = metadata["name"]
    album = metadata["album"]["name"]
    artist = metadata["album"]["artists"][0]["name"]
    release_date = metadata["album"]["release_date"]
    length = metadata["duration_ms"]
    popularity = metadata["popularity"]

    # audio features
    acousticness = features[0]["acousticness"]
    danceability = features[0]["danceability"]
    energy = features[0]["energy"]
    instrumentalness = features[0]["instrumentalness"]
    liveness = features[0]["liveness"]
    loudness = features[0]["loudness"]
    speechiness = features[0]["speechiness"]
    tempo = features[0]["tempo"]
    time_signature = features[0]["time_signature"]
    url = id

    track = [
        name,
        album,
        artist,
        release_date,
        length,
        popularity,
        danceability,
        acousticness,
        energy,
        instrumentalness,
        liveness,
        loudness,
        speechiness,
        tempo,
        time_signature,
        url,
    ]
    return track

from progress.bar import Bar

with Bar('Processing', max=total_to_scrape) as bar:

    for i, url in enumerate(target_urls):
        # print(f'Processing... {i}/{total_to_scrape} {url}')
        try:
            track = get_track_features(url)
            tracks.append(track)
        except:
            continue
        if not i % 1000:
            print('Sleeping... xoxo', flush=True)
            sleep(10)
        bar.next()

In [31]:
df_api_2 = pd.DataFrame(
    tracks,
    columns=[
        "name",
        "album",
        "artist",
        "release_date",
        "length",
        "popularity",
        "danceability",
        "acousticness",
        "energy",
        "instrumentalness",
        "liveness",
        "loudness",
        "speechiness",
        "tempo",
        "time_signature",
        "spotify_url",
    ],
)

In [32]:
df_api = pd.concat([df_api, df_api_2])

In [38]:
target_df = 'api_features'
df_api.to_pickle(f'{target_df}.pkl')

In [38]:
df_merge = pd.merge(df, df_api, left_on='spotify_url', right_on='spotify_url', how='inner')
df_merge = df_merge.drop_duplicates()
df_merge

index   streams  ranking  \
spotify_url            country week_start                             
2mnXxnrX5vCGolNkaFvVeM do      2022-12-15      0    256566        1   
                       global  2022-12-15     13  24540889       14   
                       cl      2022-12-22      3   1934347        4   
                       ar      2022-12-29     33   1236631       34   
                       cl      2022-12-08      2   2078595        3   
...                                          ...       ...      ...   
6sL1IvnaptzRRtpoUS2mz4 do      2022-12-01    192     20162      193   
1gaE7n7TXmR2xoc7zVC7Wu do      2022-12-01    197     19868      198   
0EcQcdcbQeVJn9fknj44Be ae      2022-11-17    195     12398      196   
2lHktSigIAZqf7BkA7NRS7 ca      2022-03-03     97    280204       98   
7juiva3TJwb9VmCxt17MSd ca      2022-03-03    160    225440      161   

                                                                         name  \
spotify_url            country week_start                                       
2mnXxnrX5vCGolNkaFvVeM do      2022-12-15                            La Jumpa   
                       global  2022-12-15                            La Jumpa   
                       cl      2022-12-22                            La Jumpa   
                       ar      2022-12-29                            La Jumpa   
                       cl      2022-12-08                            La Jumpa   
...                                                                       ...   
6sL1IvnaptzRRtpoUS2mz4 do      2022-12-01                          Disciplina   
1gaE7n7TXmR2xoc7zVC7Wu do      2022-12-01                       Electro vol.1   
0EcQcdcbQeVJn9fknj44Be ae      2022-11-17                    I Like Me Better   
2lHktSigIAZqf7BkA7NRS7 ca      2022-03-03  Bois Lie (feat. Machine Gun Kelly)   
7juiva3TJwb9VmCxt17MSd ca      2022-03-03                       Hard Feelings   

                                                                             album  \
spotify_url            country week_start                                            
2mnXxnrX5vCGolNkaFvVeM do      2022-12-15                               Sr. Santos   
                       global  2022-12-15                               Sr. Santos   
                       cl      2022-12-22                               Sr. Santos   
                       ar      2022-12-29                               Sr. Santos   
                       cl      2022-12-08                               Sr. Santos   
...                                                                            ...   
6sL1IvnaptzRRtpoUS2mz4 do      2022-12-01                        La Presión No Ta'   
1gaE7n7TXmR2xoc7zVC7Wu do      2022-12-01                            Electro vol.1   
0EcQcdcbQeVJn9fknj44Be ae      2022-11-17  I met you when I was 18. (the playlist)   
2lHktSigIAZqf7BkA7NRS7 ca      2022-03-03                                 Love Sux   
7juiva3TJwb9VmCxt17MSd ca      2022-03-03                            Hard Feelings   

                                                  artist release_date  length  \
spotify_url            country week_start                                       
2mnXxnrX5vCGolNkaFvVeM do      2022-12-15       Arcángel   2022-12-01  255693   
                       global  2022-12-15       Arcángel   2022-12-01  255693   
                       cl      2022-12-22       Arcángel   2022-12-01  255693   
                       ar      2022-12-29       Arcángel   2022-12-01  255693   
                       cl      2022-12-08       Arcángel   2022-12-01  255693   
...                                                  ...          ...     ...   
6sL1IvnaptzRRtpoUS2mz4 do      2022-12-01    Jhay Dohmie   2022-05-09  175331   
1gaE7n7TXmR2xoc7zVC7Wu do      2022-12-01   Pirulo Music   2022-04-28  174480   
0EcQcdcbQeVJn9fknj44Be ae      2022-11-17           Lauv   2018-05-31  197436   
2lHktSigIAZqf7BkA7NRS7 ca      2022-03-03  Avril Lavign

In [35]:
df

,index,spotify_url,week_start,streams,country,ranking
0,0,2mnXxnrX5vCGolNkaFvVeM,2022-12-15,256566,do,1
1,1,5Uq9MSXpOKkvxbk701GmYK,2022-12-15,171816,do,2
2,2,1dnlWBC8oXaxzDQWtDxies,2022-12-15,164808,do,3
3,3,6Sq7ltF9Qa7SNFBsV5Cogx,2022-12-15,148057,do,4
4,4,5ww2BF9slyYgNOk37BlC4u,2022-12-15,134607,do,5
...,...,...,...,...,...,...
270394,195,6gBFPUFcJLzWGx4lenP6h2,2022-03-03,211304,ca,196
270395,196,0I3q5fE6wg7LIfHGngUTnV,2022-03-03,210654,ca,197
270396,197,3eJH2nAjvNXdmPfBkALiPZ,2022-03-03,210431,ca,198
270397,198,7fBv7CLKzipRk6EC6TWHOB,2022-03-03,210308,ca,199


In [39]:
df_merge.to_pickle('merge.pkl')